In [2]:
import cobra
import pandas as pd
import re
import numpy as np
import scipy.stats as st
from matplotlib import pyplot as plt
from pathlib import Path
import sys
sys.path.append('../../code/')
import leakage, utils
import pubchempy as pcp
import seaborn as sns
import time


# Settings / choices

In [16]:
only_significant_changes = False
timepoints = np.arange(11.5, 32, 1)
knock_outs = False
shadow_price_for_leaked_mets = True
species = 'c_glutamicum'

In [17]:
model_fn = '../../models/{0}/smoment_iCW773.xml'.format(species)
model = cobra.io.read_sbml_model(model_fn)
# model = cobra.io.load_matlab_model('../../models/{0}/ec_iYO844.mat'.format(species))

model.solver = 'gurobi'

In [18]:
model.optimize()
print(model.summary())
model.reactions.EX_glc_e.lower_bound = -10
model.optimize()
print(model.summary())

Objective
1.0 CG_biomass_cgl_ATCC13032 = 0.434189456635201

Uptake
------
Metabolite           Reaction      Flux  C-Number  C-Flux
     btn_e          EX_BIOTIN 1.032E-06        10   0.00%
     ca2_e           EX_ca2_e   0.01467         0   0.00%
      cl_e              EX_cl   0.01467         0   0.00%
 cobalt2_e       EX_cobalt2_e 7.089E-05         0   0.00%
     cu2_e           EX_cu2_e  0.001997         0   0.00%
     fe2_e           EX_fe2_e   0.04092         0   0.00%
  glc__D_e           EX_glc_e      4.67         6 100.00%
       k_e             EX_k_e    0.5499         0   0.00%
     mg2_e           EX_mg2_e   0.02444         0   0.00%
     mn2_e           EX_mn2_e  0.001948         0   0.00%
     nh4_e           EX_nh4_e     3.758         0   0.00%
     ni2_e           EX_ni2_e 0.0009093         0   0.00%
      o2_e            EX_o2_e      8.78         0   0.00%
      pi_e            EX_pi_e    0.2157         0   0.00%
     so4_e           EX_so4_e   0.06962         0   0.00

In [19]:
def get_leakage(time):
    exometabolites_folder = Path("../../data/{0}/".format(species))
    leakage_df = leakage.get_leakage(exometabolites_folder, species, time = time, unit = '/gDW', method = 'one-way-diff',
                                    only_significant_changes = only_significant_changes)
    leakage_df.set_index("Metabolite", inplace=True)
    leakage_df.drop_duplicates(inplace=True)
    leakage_label = "Leakage (mmol/gDW/h)"
    return leakage_df

In [20]:
# # Filenames
# organism = 'c_glutamicum'
# fn_exometabolites = exometabolites_folder / "{0}_exometabolites.csv".format(organism)
# fn_exometabolites_std = exometabolites_folder / "{0}_exometabolites_std.csv".format(organism)
# fn_OD = exometabolites_folder / "{0}_OD.csv".format(organism)

# # Read files as dataframes
# df_exometabolites = pd.read_csv(fn_exometabolites, index_col=0)
# df_exometabolites_std = pd.read_csv(fn_exometabolites_std, index_col=0)
# df_OD = pd.read_csv(fn_OD, index_col=0)
    

In [21]:
exometabolites_folder = Path("../../data/{0}".format(species))


In [22]:
met_info_df = pd.read_csv("../../data/met_info_curated.csv", encoding = "ISO-8859-1", index_col = 0)

In [23]:
# Read metabolite mapping
mapping_df = pd.read_csv('../../data/id_mapping.csv', index_col=0)
# df2 = pd.merge(leakage_df, mapping_df, left_index=True, right_index=True)
# df2.drop(columns='Metabolite name', inplace=True)

# Get leakage


In [24]:
# timepoints = np.arange(1.5, 12, 1)#[5,6,7,8,9,10,11, 12, 13]
for i, t in enumerate(timepoints):
    print(t)
    leakage_df = get_leakage(t)
    # Consider to use an earlier time-point
    glucose_uptake_rate = leakage.get_glucose_uptake_rate(exometabolites_folder, species, time = t, method = 'one-way-diff')
    df2 = pd.merge(leakage_df, mapping_df, left_index=True, right_index=True)
    df2.drop(columns='Metabolite name', inplace=True)
    df = pd.merge(met_info_df, df2, left_on = 'Metabolite id', right_on = 'Bacillus metabolite')
    df['Time'] = t
    df['Glucose'] = -glucose_uptake_rate
    print(glucose_uptake_rate)
    # Set model constraints
    model = cobra.io.read_sbml_model(model_fn)
    with model:
        model.reactions.EX_glc_e.lower_bound = min(glucose_uptake_rate, 0)
        print(model.slim_optimize())
        for j, row in df.iterrows():
            if row['Leakage (mmol/gDW/h)'] < 0:
                met_ids = row['Metabolite id'].split(',')
                mets = []
                for m_id in met_ids:
                    try:
                        m = model.metabolites.get_by_id('{0}_e'.format(m_id.strip(' ')))
                    except KeyError:
                        continue
                    else:
                        mets.append(m)
                for m in mets:
                    try:
                        r_ex = [r for r in m.reactions if len(r.metabolites)==1][0]
                    except IndexError:
                        print(m.id, [r for r in m.reactions])
                    else:
                        r_ex.lower_bound = row['Leakage (mmol/gDW/h)']/len(mets)
        solution = model.optimize()
        # print(model.summary())
        try:
            pfba_solution = cobra.flux_analysis.pfba(model.copy())
        except:
            print('No feasible model at t: ', t)
            continue
        print(model.slim_optimize())
        # List already excreted metabolites
        exchanged_mets = {}
        for r in model.boundary:
            flux = solution.fluxes[r.id]
            if flux != 0:
                exchanged_mets[list(r.metabolites.keys())[0].id[:-2]]=(r.id, flux)
        
        df['Predicted growth rate'] = solution.objective_value
        # Get turnover and shadow prices
        turnover = {}
        shadow_prices = {}        
        for j, row in df.iterrows():
            if row['Leakage (mmol/gDW/h)'] > 0:
                met_ids = row['Metabolite id'].split(',')
                sp_list = []
                turnover_list = []
                for key in met_ids:
                    if key.strip() in exchanged_mets.keys():
                        if shadow_price_for_leaked_mets:
                            # the change should be taken into account the already exchanged flux
                            existing_flux = exchanged_mets[key.strip()]
                        else:
                            continue
                    else:
                        existing_flux = None
                    m_id = "{0}_c".format(key.strip())
                    m = model.metabolites.get_by_id(m_id)
                    # x = solution.objective_value
                    # x2 = model.optimize().objective_value
                    sp_ji = leakage.estimate_shadow_price_for_met(model, m, solution, delta = 0.01, existing_flux = existing_flux)
                    if np.abs(sp_ji-solution.shadow_prices[m.id]) > 0.01:
                        sp_ji = solution.shadow_prices[m.id]
                        # Need to figure out what's wrong here
                            # solution = M.optimize()
                        print("######")
                            # print(x, x2, solution.objective_value)
                            # print(m.id, sp_ji, solution.shadow_prices[m.id], existing_flux)
                    sp_list.append(sp_ji)
                    # turnover_list.append(get_turnover_flux(m, pfba_solution))
                    turnover_list.append(m.summary(pfba_solution).producing_flux['flux'].sum())
                # print(met_ids, sp_list, turnover_list)
                # Shadow prices
                if len(sp_list):
                    shadow_prices[j] = np.nanmean(sp_list)
                    turnover[j] = np.mean(turnover_list)
                else:
                    shadow_prices[j] = np.nan
                    turnover[j] = np.nan
                # print(met_ids, np.nanmean(sp_list))
        df["Shadow price"] = pd.Series(shadow_prices)
        df["Turnover"] = pd.Series(turnover)
        
    if i == 0:
        full_df = df
    else:
        full_df = pd.concat([full_df, df])


11.5
-7.513987495685255
0.4799282972318963
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpjnvovcfs.lp
Reading time = 0.01 seconds
: 950 rows, 3666 columns, 17628 nonzeros
0.4799282972318963
12.5
-5.104239728624449
0.4554560788591469
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpcmvp3mts.lp
Reading time = 0.01 seconds
: 950 rows, 3666 columns, 17628 nonzeros
0.45546398170154667
13.5
-1.580257091688144
0.14692311948265493
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpsm2sqwh1.lp
Reading time = 0.01 seconds
: 950 rows, 3666 columns, 17628 nonzeros
0.14692311948265493
14.5
-2.6172567560265096
0.24333732093660781
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpbhmrdugy.lp
Reading time = 0.01 seconds
: 950 rows, 3666 columns, 17628 nonzeros
0.24333732093660781
15.5
-2.3905348659214702
0.22225803736658356
Read LP format model from file /var/folders/h

/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp

20.5
0.0
0.0
r5p_e []
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmp4xomuhif.lp
Reading time = 0.01 seconds
: 950 rows, 3666 columns, 17628 nonzeros
0.0


/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp

21.5
0.0
0.0
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpull3_c2k.lp
Reading time = 0.01 seconds
: 950 rows, 3666 columns, 17628 nonzeros
1.3789623706105106e-06
22.5
0.0


...

/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp

0.0
r5p_e []
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmptxd1fwr5.lp
Reading time = 0.01 seconds
: 950 rows, 3666 columns, 17628 nonzeros
5.073809647732357e-05
31.5
0.0


/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp

0.0
r5p_e []
Read LP format model from file /var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/tmpv_gbcgot.lp
Reading time = 0.01 seconds
: 950 rows, 3666 columns, 17628 nonzeros
3.730281056144042e-05


/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp/T/ipykernel_82822/152855143.py:87: RuntimeWarning: Mean of empty slice
  shadow_prices[j] = np.nanmean(sp_list)
/var/folders/h6/4b_zz_cd5d92w2ycp017ytn00000gp

In [25]:
full_df

,Metabolite name,Metabolite id,Value,Uncertainty,Mass,Charge,Phosphate,Topological Polar Surface Area [],Concentration,log P,Leakage (mmol/gDW/h),Ecoli metabolite,Yeast metabolite,Bacillus metabolite,Time,Glucose,Predicted growth rate,Shadow price,Turnover
0,Dihydroxyacetonephosphate,dhap,50.40,1.74,168.041961,-2.0,1,104.0,0.000374,-2.5,0.000424,dhap,s_0629,dhap,11.5,7.513987,0.479928,-0.008836,4.612112
1,Glyceraldehyde-3-phosphate,g3p,15.04,0.24,168.041961,-2.0,1,104.0,NaN,-2.7,0.000027,g3p,s_0764,g3p,11.5,7.513987,0.479928,-0.008841,11.037761
2,2/3-phosphoglycerate,"2pg, 3pg",19.45,1.55,183.033421,-3.0,1,124.0,0.001540,-2.6,0.000311,"2pg, 3pg","s_0188, s_0260","2pg, 3pg",11.5,7.513987,0.479928,-0.009413,9.787341
3,Ribose-5-phosphate,r5p,10.29,0.22,228.093921,-2.0,1,137.0,0.000450,-3.6,0.000032,r5p,s_1408,r5p,11.5,7.513987,0.479928,-0.008162,1.462941
4,Erythrose-4-phosphate,e4p,1.31,0.21,198.067941,-2.0,1,124.0,NaN,-3.3,-0.000055,e4p,s_0551,e4p,11.5,7.513987,0.479928,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,Glutamate,glu__L,0.00,0.00,146.121320,-1.0,0,101.0,0.096000,-3.7,0.000603,glu__L,s_0991,glu__L,31.5,-0.000000,0.000037,NaN,0.003731
13,Glutamine,gln__L,0.00,0.00,146.144500,0.0,0,106.0,0.003810,-3.1,0.000005,gln__L,s_0999,gln__L,31.5,-0.000000,0.000037,NaN,0.001792
14,Proline,pro__L,0.54,0.13,115.130460,0.0,0,49.3,0.000385,-2.5,NaN,pro__L,s_1035,pro__L,31.5,-0.000000,0.000037,NaN,NaN
15,Serine,ser__L,0.87,0.52,105.092580,0.0,0,83.6,0.000068,-3.1,0.000182,ser__L,s_1039,ser__L,31.5,-0.000000,0.000037,NaN,0.000992


# Add median shadow prices


In [26]:
# full_leakage['Uptake (mmol/gDW/h)'] = 0
new_df = full_df.copy()

In [27]:
# median_sp_df = pd.read_csv('../../results/e_coli/median_sp.csv', index_col = 0)

In [28]:
# median_sp = []
# low_sp = []
# high_sp = []
# for i, row in new_df.iterrows():
    
#     gr = np.round(row['Predicted growth rate'], 1)
    
#     tmp_m = []
#     tmp_low = []
#     tmp_high = []
#     for key in row['Ecoli metabolite'].split(','):
#         met_id = '{0}_c'.format(key.strip())
#         met_df = median_sp_df.loc[met_id,:]
#         tmp_m.append(met_df.loc[met_df['Growth rate']==gr, 'log10(-Shadow price)'].values[0])
#         tmp_low.append(met_df.loc[met_df['Growth rate']==0.2, 'log10(-Shadow price)'].values[0])
#         tmp_high.append(met_df.loc[met_df['Growth rate']==0.5, 'log10(-Shadow price)'].values[0])
#     median_sp.append(np.mean(tmp_m))
#     low_sp.append(np.mean(tmp_low))
#     high_sp.append(np.mean(tmp_high))

In [29]:
new_df['Median log10(-Shadow price)'] = median_sp
new_df['Low log10(-Shadow price)'] = low_sp
new_df['High log10(-Shadow price)'] = high_sp

NameError: name 'median_sp' is not defined

In [30]:
# np.round?

In [31]:
# full_leakage['Uptake (mmol/gDW/h)'] = 0
# new_df = full_df.copy()

In [32]:
new_df['Uptake (mmol/gDW/h)'] = -1*new_df['Leakage (mmol/gDW/h)']

In [33]:
new_df.loc[new_df['Leakage (mmol/gDW/h)'] < 0, 'Leakage (mmol/gDW/h)'] = np.nan
new_df.loc[new_df['Uptake (mmol/gDW/h)'] < 0, 'Uptake (mmol/gDW/h)'] = 0


In [34]:
new_df['log10(Leakage [mmol/gDW/h])'] = np.log10(new_df['Leakage (mmol/gDW/h)'])
new_df['log10(-Shadow price [gDW/mmol])'] = np.log10(-new_df['Shadow price'])
new_df['log10(Turnover [mmol/gDW/h])'] = np.log10(new_df['Turnover']).replace(-np.inf, np.nan)

/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/ssulheim/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [35]:
new_df.drop(columns=['Value', 'Uncertainty', 'Yeast metabolite', 'Ecoli metabolite', 'Bacillus metabolite'], inplace = True)

In [36]:
timestr = time.strftime("%Y%m%d")

if only_significant_changes:
    s1 = '_osc'
else:
    s1 = ''

if shadow_price_for_leaked_mets:
    s2 = '_SP_for_leaked'
else:
    s2 = ''

if knock_outs:
    s3 = '_KO'
else:
    s3 = ''
fn = 'spreadsheet_{0}_leakage_{1}{2}{3}{4}.csv'.format(species, timestr, s1, s2, s3)
folder = Path('../../results/{0}/'.format(species))
new_df.to_csv(folder / fn)

In [23]:
# full_leakage = full_df.loc[~full_df['Shadow price'].isna(), :]
# full_leakage = full_leakage.loc[full_leakage.Turnover <100, :]
# full_leakage = full_leakage.loc[full_leakage['Leakage (mmol/gDW/h)'] > 0, :]

In [37]:
timestr

'20230717'

In [24]:
# full_leakage['log10(leakage)'] = np.log10(full_leakage['Leakage (mmol/gDW/h)'])
# full_leakage['log10(-Shadow price)'] = np.log10(-full_leakage['Shadow price'])
# full_leakage['log10(Turnover)'] = np.log10(full_leakage['Turnover'])